In [101]:
#library loading for preprocessing
import re
import os
import sys
import json

from pykospacing import spacing #한국어 전처리_의미에 맞게 띄어쓰기 정리해주는 것
from hanspell import spell_checker #한국어 전처리_한글 맞춤법 검사기
import kss
from konlpy.tag import Okt #okt는 형태소 추출기 okt.morphs(txt)
from konlpy.tag import Kkma #konlpy는 형태소 분석기
from ckonlpy.tag import Twitter #customized konlpy
twitter=Twitter()

kkma = Kkma()
okt = Okt()

import nltk #nltk.download()
from nltk.corpus import stopwords #불용어 제거
from nltk.tokenize import word_tokenize #불용어 제거

In [103]:
#read review files
iphone12_reviews = []
for i in range(10):
    f = open("data" + str(i+1) + ".txt", "r")
    lines = f.readlines()
    raw_data = ""
    for line in lines:
        raw_data += line
    iphone12_reviews.append(raw_data)

In [104]:
#cleansing function
def cleansing(review):
    review = re.sub(r'[0-9]+:[0-9]+', '', review) #remove timestamp
    review = re.sub(r'\s+', ' ', review)
    review = review.replace(" ", "")
    review = spacing(review)
    review_result = ''
    for sentence in kss.split_sentences(review):
        sentence = spell_checker.check(sentence) #한글 맞춤법 검사기
        sentence = sentence.checked
        if sentence[-1] == '?':
            sentence += ' '
        elif sentence[-1] != '.':
            sentence += '. '
        else:
            sentence += ' '
        review_result += sentence
    return review_result

In [105]:
cleansed_reviews =[]
for review in iphone12_reviews:
    review = cleansing(review)
    cleansed_reviews.append(review)

In [106]:
cleansed_reviews[0]

'티셔츠 공부할게요. 시연 공개합니다. 안녕하세요. 안녕하세요 비 쎌입니다. 이렇게 관심도가 높고 인기가 많을 줄은 전혀 예상치 못했습니다. 이때까지 출시된 아이폰 중에서 역대급이라고 할 수 없지만 아이폰 11이 출시되었을 때보다 훨씬 더 많은 분들이 관심을 갖고 있는 제품이 아이폰 2를 보인 것 같습니다. 실제로 공개된 경쟁사 대비 스펙이 엄청나게 뛰어나다. 이렇게 말할 것은 없었지만 디자인이 바뀐 적이 가장 큰 것 같습니다. 게 다가 요즘에는 아이폰이나 갤럭시를 많이 번갈아 가면서 쓰기보다는 갤럭시의 팬들은 새로운 갤럭시를 원하고 아이폰 주자분들은 새로운 아이폰만을 원하기 때문에. 기존 많은 아이폰 유저분들이 아이폰 2를 기다린 것 같아요. 그래서 이상하게 주위에 관심도가 너무 많다. 보니까 나도 이거를 빨리 사야 하나라는 생각에 일단 홍콩 직구 판로 질렀습니다. 보미는 배송만 줄 뿐 원래 같으면 갤럭시 S2N EFE처럼 조금 써보고 리뷰로 올리려고 했으나 지금 뻥 덩이가 실로 끓여서 안 되기 때문에 간단히 엇박심을 하고 아이폰 일레븐 프로에 어떠한 차이점이 있는지 이 정도만 간단히 비교를 해볼게 요 박스로 처음 받았을 때 진짜 이게 뭐 야 장난감인가? 이런 생각이 들었습니다. 왜냐하면 지난번 출시되었던 이 폰 11 프로박스와 비교했을 때 그 차이는 많이 나지 않아요. 하지만 이곳을 옆으로 돌려봤을 때 진짜 엄청 크게. 박스에 사이즈가 반으로 줄어듭니다. 그 사이즈가 반으로 줄어듭니다. 이 사이즈가 반으로 줄어듭니다. 그렇기 때문에. 뭔가 장난감을 들고 있는 듯한 느낌. 하지만 박스를 처음에 들었을 때 약간 묵직한 것을 느껴졌고. 애플답지 않게 박스를 흔들었을 때 안에서 구성품이 흔들리는. 달랑 달랑거리는 소리가 났습니다. 예전에 LG 스마트폰 도 달랑 달랑거리는 소리가 났는데 제가 처음 경험을 해보는 것 같아요. 일단 제가 구입한 것은 홍콩 직구 모델로 구입했고. 가격은 한국보다 훨씬 더 비싸게 구입했어요. 특히 이제 한국 아이폰 가격이 이제는 그렇게 비

In [107]:
twitter.morphs(cleansed_reviews[0]) #형태소 추출
#okt.pos(cleansed_reviews[1]) #품사태깅(part of speech tagging)
#okt.nouns(cleansed_reviews[1])

['티셔츠',
 '공부',
 '할게요',
 '.',
 '시연',
 '공개',
 '합니다',
 '.',
 '안녕',
 '하세요',
 '.',
 '안녕',
 '하세요',
 '비',
 '쎌',
 '입니다',
 '.',
 '이렇게',
 '관심',
 '도',
 '가',
 '높고',
 '인기',
 '가',
 '많을',
 '줄',
 '은',
 '전혀',
 '예상',
 '치',
 '못',
 '했습니다',
 '.',
 '이',
 '때',
 '까지',
 '출시',
 '된',
 '아이폰',
 '중',
 '에서',
 '역대',
 '급',
 '이라고',
 '할',
 '수',
 '없',
 '지만',
 '아이폰',
 '11',
 '이',
 '출시',
 '되었을',
 '때',
 '보다',
 '훨씬',
 '더',
 '많은',
 '분',
 '들',
 '이',
 '관심',
 '을',
 '갖고',
 '있는',
 '제품',
 '이',
 '아이폰',
 '2',
 '를',
 '보인',
 '것',
 '같습니다',
 '.',
 '실제',
 '로',
 '공개',
 '된',
 '경쟁',
 '사',
 '대비',
 '스펙',
 '이',
 '엄청',
 '나게',
 '뛰어나다',
 '.',
 '이렇게',
 '말',
 '할',
 '것',
 '은',
 '없었',
 '지만',
 '디자인',
 '이',
 '바뀐',
 '적',
 '이',
 '가장',
 '큰',
 '것',
 '같습니다',
 '.',
 '게',
 '다가',
 '요즘',
 '에는',
 '아이폰',
 '이나',
 '갤럭시',
 '를',
 '많이',
 '번갈아',
 '가면',
 '서',
 '쓰기',
 '보다는',
 '갤럭시',
 '의',
 '팬',
 '들',
 '은',
 '새',
 '로운',
 '갤럭시',
 '를',
 '원',
 '하고',
 '아이폰',
 '주자',
 '분',
 '들',
 '은',
 '새',
 '로운',
 '아이폰',
 '만을',
 '원하기',
 '때문',
 '에',
 '.',
 '기존',
 '많은',
 '아이폰',
 '유저',
 '분',
 '들',

In [108]:
stopwords_file = open("한국어불용어100.txt", "r")
stop_words = []
for line in stopwords_file.readlines():
    stop_words.append(line)
stopwords_file.close()

In [111]:
word_tokens = word_tokenize(cleansed_reviews[0])
result = []
for w in word_tokens:
    if w not in stop_words:
        result.append(w)
print(result)

['티셔츠', '공부할게요', '.', '시연', '공개합니다', '.', '안녕하세요', '.', '안녕하세요', '비', '쎌입니다', '.', '이렇게', '관심도가', '높고', '인기가', '많을', '줄은', '전혀', '예상치', '못했습니다', '.', '이때까지', '출시된', '아이폰', '중에서', '역대급이라고', '할', '수', '없지만', '아이폰', '11이', '출시되었을', '때보다', '훨씬', '더', '많은', '분들이', '관심을', '갖고', '있는', '제품이', '아이폰', '2를', '보인', '것', '같습니다', '.', '실제로', '공개된', '경쟁사', '대비', '스펙이', '엄청나게', '뛰어나다', '.', '이렇게', '말할', '것은', '없었지만', '디자인이', '바뀐', '적이', '가장', '큰', '것', '같습니다', '.', '게', '다가', '요즘에는', '아이폰이나', '갤럭시를', '많이', '번갈아', '가면서', '쓰기보다는', '갤럭시의', '팬들은', '새로운', '갤럭시를', '원하고', '아이폰', '주자분들은', '새로운', '아이폰만을', '원하기', '때문에', '.', '기존', '많은', '아이폰', '유저분들이', '아이폰', '2를', '기다린', '것', '같아요', '.', '그래서', '이상하게', '주위에', '관심도가', '너무', '많다', '.', '보니까', '나도', '이거를', '빨리', '사야', '하나라는', '생각에', '일단', '홍콩', '직구', '판로', '질렀습니다', '.', '보미는', '배송만', '줄', '뿐', '원래', '같으면', '갤럭시', 'S2N', 'EFE처럼', '조금', '써보고', '리뷰로', '올리려고', '했으나', '지금', '뻥', '덩이가', '실로', '끓여서', '안', '되기', '때문에', '간단히', '엇박심을', '하고', '아이폰', '일레븐', '프로에', '어떠한', '차이

In [160]:
#customized sentiment dictionary for sentimental analysis
#original categories: 서비스 / 맛 / 가격 / 분위기 / 방문

color_feature = {'색':['로즈골드', '퍼플', '레드', '유광', '무광'],
                '컬러':['로즈골드', '퍼플', '레드', '유광', '무광'],
                '칼라':['로즈골드', '퍼플', '레드', '유광', '무광']}
color_good_feature = {'색':['고급', '좋', '괜찮', '최고', '짱', '훌륭', '추천', '최고', '만족', '세련', '최고', '감동', '특별', '새로운'],
                     '컬러':['고급', '좋', '괜찮', '최고', '짱', '훌륭', '추천', '최고', '만족', '세련', '최고', '감동', '특별', '새로운'],
                     '칼라':['고급', '좋', '괜찮', '최고', '짱', '훌륭', '추천', '최고', '만족', '세련', '최고', '감동', '특별', '새로운']}
color_bad_feature = {'색':['별로', '최악', '질리'],
                    '컬러':['별로', '최악', '질리'],
                    '칼라':['별로', '최악', '질리']}

size_big_feature = {'크기':['크','큰','시원', '부담'], '사이즈':['크','큰','시원', '부담']}
size_small_feature = {'크기':['작', '아담', '조그', '쪼그', '미니', '스몰'], '사이즈':['작', '아담', '조그', '쪼그', '미니', '스몰']}

cap_big_feature = {'용량':['크','큰'], '기가':['크','큰']}
cap_small_feature = {'용량':['작', '적'], '기가':['작', '적']}

weight_heavy_feature = {'무게':['증가', '무거', '무겁']}
weight_light_feature = {'무게':['감소', '가벼', '가볍']}

display_good_feature = {'디스플레이':['만족', '괜찮', '얇'], '베젤':['만족', '괜찮', '얇']}
display_bad_feature = {'디스플레이':['별로', '두껍', '거슬'], '베젤':['별로', '두껍', '거슬']}

price_good_feature = {'가격': ['괜찮','착하다','저렴','적당','싸다','좋다','합리적','훌륭','최고','만족','알맞다', '무난','괜춘','최상','최상','굿', '추천', '뛰어난'],
                     '가성비':['괜찮','착하다','저렴','적당','싸다','좋다','합리적','훌륭','최고','만족','알맞다', '무난','괜춘','최상', '굿', '추천', '뛰어난']}
price_bad_feature ={'가격':['비싸','있는','있다','나쁘','사악','비효율','높다','부담','아쉽','별로','그닥','그다지','쎄','높','거품'],
                   '가성비':['비싸','있는','있다','나쁘','사악','비효율','높다','부담','아쉽','별로','그닥','그다지','쎄','높','거품']}

charge_good_feature = {'배터리':['만족','괜찮','상향','대폭','오래','길','충분','빠른'],
                      '충전':['만족','괜찮','상향','대폭','오래','길','충분','빠른']}
charge_bad_feature = {'배터리':['별로','부족'],
                     '충전':['별로','부족']}

negative_word_emotion = ['안','않','못','없','아닌','아니']

In [107]:
#특징 키워드를 추출하는 함수
#특징 키워드가 속한 문장에서 뒤로 단어 + 키워드 + 단어 / 키워드 + 단어 + 단어 를 추출
def get_feature_keywords(feature_keywords, review):
    feature_temp = []
    for keyword in feature_keywords:
        if re.findall(keyword, review):
            sub_list = ['게','고','음','며','데','만','도','면']
            
            for sub in sub_list:
                if sub+' ' in review:
                    review = re.sub(sub+' ', sub+',', review)
                
            a = re.findall(keyword +'+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+',review) # K한 한 한글
            b = re.findall(keyword + '+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+',review) # K 한 한글 
            c = re.findall('[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+' + keyword +'[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+',review) # 한 K한글 예쁜 분위기가
                
            for ngram in a:
                t = ()
                feature_temp.append(t + (ngram,keyword))
            for ngram in b:
                t = ()
                feature_temp.append(t + (ngram,keyword))
            for ngram in c:
                t = ()
                feature_temp.append(t + (ngram,keyword))     
    return feature_temp

In [108]:
def get_feature_emotions(feature_good_dict,feature_bad_dict,feature_temp):
    good_feature_emotion_list = []
    bad_feature_emotion_list = []
    
    for ngrams in feature_temp:
        keyword = ngrams[1]
        ngram = ngrams[0]
        is_bad_feature = None
        
        good_emotion_list = feature_good_dict[keyword]
        bad_emotion_list = feature_bad_dict[keyword]
        for emotion in good_emotion_list:
            if re.findall(emotion, ngram):
                is_bad_feature = False  
        for emotion in bad_emotion_list:
            if re.findall(emotion, ngram):
                is_bad_feature = True    
        for negative in negative_word_emotion:
            if re.findall(negative, ngram):
                if is_bad_feature == True:
                    is_bad_feature = False
                    break
                elif is_bad_feature == False:
                    is_bad_feature = True
                    break
                else:
                    is_bad_feature = True
                    break   
        if is_bad_feature:
            bad_feature_emotion_list.append(ngram)
        elif is_bad_feature == False:
            good_feature_emotion_list.append(ngram)
        else:
            pass
    return good_feature_emotion_list, bad_feature_emotion_list

In [166]:
good_color_cnt = 0
bad_color_cnt = 0
big_size_cnt = 0
small_size_cnt = 0
big_cap_cnt = 0
small_cap_cnt = 0
heavy_cnt = 0
light_cnt = 0
good_display_cnt = 0
bad_display_cnt = 0
good_price_cnt = 0
bad_price_cnt = 0

for review in cleansed_reviews: 
    color_temp = get_feature_keywords(color_good_feature.keys(), review)
    good_color, bad_color = get_feature_emotions(color_good_feature, color_bad_feature, color_temp)

    size_temp = get_feature_keywords(size_big_feature.keys(), review)
    big_size, small_size = get_feature_emotions(size_big_feature, size_small_feature, size_temp)
    
    cap_temp = get_feature_keywords(cap_big_feature.keys(), review)
    big_cap, small_cap = get_feature_emotions(cap_big_feature, cap_small_feature, cap_temp)
    
    weight_temp = get_feature_keywords(weight_heavy_feature.keys(), review)
    heavy_weight, light_weight = get_feature_emotions(weight_heavy_feature, weight_light_feature, weight_temp)
    
    display_temp = get_feature_keywords(display_good_feature.keys(), review)
    good_display, bad_display = get_feature_emotions(display_good_feature, display_bad_feature, display_temp)
    
    price_temp = get_feature_keywords(price_good_feature.keys(), review)
    good_price, bad_price = get_feature_emotions(price_good_feature, price_bad_feature, price_temp)
    
if len(good_color) > len(bad_color):
    good_color_cnt += 1
elif len(good_color) < len(bad_color):
    bad_color_cnt +=1
else:
    pass

if len(big_size) > len(small_size):
    big_size_cnt += 1
elif len(big_size) < len(small_size):
    small_size_cnt +=1
else:
    pass

if len(big_cap) > len(small_cap):
    big_cap_cnt += 1
elif len(big_cap) < len(small_cap):
    small_cap_cnt +=1
else:
    pass

if len(heavy_weight) > len(light_weight):
    heavy_cnt += 1
elif len(heavy_weight) < len(light_weight):
    light_cnt +=1
else:
    pass

if len(good_display) > len(bad_display):
    good_display_cnt += 1
elif len(good_display) < len(bad_display):
    bad_display_cnt +=1
else:
    pass

if len(good_price) > len(bad_price):
    good_price_cnt += 1
elif len(good_price) < len(bad_price):
    bad_price_cnt +=1
else:
    pass

In [169]:
check_division = lambda x, y: y if y ==0 else round((x / float(y)),2)
print('color is good? {}/{} = {}%'.format(
    good_color_cnt, good_color_cnt + bad_color_cnt,
    100*check_division(good_color_cnt, good_color_cnt + bad_color_cnt)))
print('size is big? {}/{} = {}%'.format(
    big_size_cnt, big_size_cnt + small_size_cnt,
    100*check_division(big_size_cnt, big_size_cnt + small_size_cnt)))
print('capacity is big? {}/{} = {}%'.format(
    big_cap_cnt, big_cap_cnt + small_cap_cnt,
    100*check_division(big_cap_cnt, big_cap_cnt + small_cap_cnt)))
print('weight is heavy? {}/{} = {}%'.format(
    heavy_cnt, heavy_cnt + light_cnt,
    100*check_division(heavy_cnt, heavy_cnt + light_cnt)))
print('display is good? {}/{} = {}%'.format(
    good_display_cnt, good_display_cnt + bad_display_cnt,
    100*check_division(good_display_cnt, good_display_cnt + bad_display_cnt)))
print('price is good? {}/{} = {}%'.format(
    good_price_cnt, good_price_cnt + bad_price_cnt,
    100*check_division(good_price_cnt, good_price_cnt + bad_price_cnt)))

color is good? 1/1 = 100.0%
size is big? 1/1 = 100.0%
capacity is big? 0/1 = 0.0%
weight is heavy? 0/0 = 0%
display is good? 1/1 = 100.0%
price is good? 0/1 = 0.0%
